In [5]:
!pip install xlrd>=2.0.1 openpyxl>=3.0.0 paramiko>=2.7.0 pandas>=1.0.0 xlsxwriter>=1.0.0

In [15]:
import pandas as pd
import zipfile
import os
from pathlib import Path
from datetime import datetime
import importlib
import sys

def check_dependencies():
    """Check and install required dependencies"""
    required_packages = {
        'xlrd': 'xlrd>=2.0.1',
        'openpyxl': 'openpyxl>=3.0.0',
        'pandas': 'pandas>=1.0.0',
        'xlsxwriter': 'xlsxwriter>=1.0.0'
    }
    
    missing_packages = []
    for package, version in required_packages.items():
        if importlib.util.find_spec(package) is None:
            missing_packages.append(version)
    
    if missing_packages:
        print("Missing required packages. Installing...")
        try:
            import pip
            for package in missing_packages:
                pip.main(['install', package])
            print("Dependencies installed successfully!")
            
            # Reload pandas after installing dependencies
            importlib.reload(pd)
        except Exception as e:
            print(f"Error installing dependencies: {str(e)}")
            print("\nPlease install the following packages manually:")
            print("\npip install " + " ".join(missing_packages))
            sys.exit(1)

def read_excel_file(file_path):
    """
    Read Excel file with proper engine based on file extension
    """
    # Convert Path object to string and get lowercase version for comparison
    file_path_str = str(file_path)
    file_lower = file_path_str.lower()
    
    try:
        if file_lower.endswith('.xls'):
            # For old .xls files
            return pd.read_excel(file_path_str, engine='xlrd', header=None)
        elif file_lower.endswith('.xlsx'):
            # For newer .xlsx files
            return pd.read_excel(file_path_str, engine='openpyxl', header=None)
        else:
            raise ValueError(f"Unsupported file format: {file_path_str}")
    except Exception as e:
        raise Exception(f"Error reading file {file_path_str}: {str(e)}")

def extract_data_from_excel_files(zip_path, output_format='xlsx'):
    """
    Extract multiple fields from Excel files within a ZIP archive and save to a new file.
    
    Args:
        zip_path (str): Path to the ZIP file containing Excel sheets
        output_format (str): 'xlsx' or 'csv' for output format
        
    Returns:
        str: Path to the created output file
    """
    # Check dependencies first
    check_dependencies()
    
    results = []
    
    # Create a temporary directory to extract files
    temp_dir = "temp_excel_files"
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # Extract all files from ZIP
        print(f"Extracting files from: {zip_path}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        
        # Process each Excel file
        excel_files = []
        for ext in ['*.xls', '*.xlsx']:
            excel_files.extend(list(Path(temp_dir).glob(ext)))
            
        if not excel_files:
            print("No Excel files found in the ZIP archive")
            return None
            
        for file_path in excel_files:
            print(f"Processing: {file_path.name}")
            try:
                # Read the Excel file with appropriate engine
                df = read_excel_file(file_path)
                
                # Extract required cells (adjusting for 0-based indexing)
                company = df.iloc[14, 5] if df.shape[0] > 14 and df.shape[1] > 5 else None  # F15
                contact_name = df.iloc[16, 5] if df.shape[0] > 16 and df.shape[1] > 5 else None  # F17
                last_name = df.iloc[17, 5] if df.shape[0] > 17 and df.shape[1] > 5 else None  # F18
                email = df.iloc[19, 5] if df.shape[0] > 19 and df.shape[1] > 5 else None  # F20
                email2 = df.iloc[20, 5] if df.shape[0] > 20 and df.shape[1] > 5 else None  # F21
                
                results.append({
                    'File Name': file_path.name,
                    'Company': company,
                    'Contact Name': contact_name,
                    'Last Name': last_name,
                    'Email': email,
                    'Email2': email2
                })
                
            except Exception as e:
                print(f"Error processing {file_path.name}: {str(e)}")
                results.append({
                    'File Name': file_path.name,
                    'Company': f'Error: {str(e)}',
                    'Contact Name': 'Error',
                    'Last Name': 'Error',
                    'Email': 'Error',
                    'Email2': 'Error'
                })
                
    finally:
        # Clean up temporary files
        print("Cleaning up temporary files...")
        for file in Path(temp_dir).glob('*'):
            try:
                file.unlink()
            except:
                pass
        try:
            os.rmdir(temp_dir)
        except:
            pass
    
    if not results:
        print("No data was extracted from any files")
        return None
        
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Generate output filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"extracted_data_{timestamp}"
    
    # Save to specified format
    if output_format.lower() == 'csv':
        output_path = f"{output_filename}.csv"
        results_df.to_csv(output_path, index=False)
    else:  # xlsx
        output_path = f"{output_filename}.xlsx"
        print(f"Creating Excel file: {output_path}")
        # Create Excel writer with better formatting
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            results_df.to_excel(writer, index=False, sheet_name='Extracted Data')
            
            # Get workbook and worksheet objects for formatting
            workbook = writer.book
            worksheet = writer.sheets['Extracted Data']
            
            # Add some formatting
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#D3D3D3',
                'border': 1
            })
            
            # Apply formatting to header row
            for col_num, value in enumerate(results_df.columns.values):
                worksheet.write(0, col_num, value, header_format)
                
            # Adjust column widths
            for idx, col in enumerate(results_df.columns):
                max_length = max(
                    results_df[col].astype(str).apply(len).max(),
                    len(col)
                ) + 2
                worksheet.set_column(idx, idx, max_length)
    
    print(f"Data extracted successfully to: {output_path}")
    return output_path

# Usage example
if __name__ == "__main__":
    # First install required dependencies if needed
    check_dependencies()
    
    zip_path = r"D:\OneDrive - Kingsford (Thailand) Co., Ltd\JLQuotations"

    # Create Excel file (default)
    print("\nCreating Excel output...")
    excel_output = extract_data_from_excel_files(zip_path)
    if excel_output:
        print(f"Excel file created: {excel_output}")

    # Optionally create CSV file
    print("\nCreating CSV output...")
    csv_output = extract_data_from_excel_files(zip_path, output_format='csv')
    if csv_output:
        print(f"CSV file created: {csv_output}")


Creating Excel output...
Extracting files from: D:\OneDrive - Kingsford (Thailand) Co., Ltd\JLQuotations
Cleaning up temporary files...


PermissionError: [Errno 13] Permission denied: 'D:\\OneDrive - Kingsford (Thailand) Co., Ltd\\JLQuotations'

In [17]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime
import importlib
import sys

def check_dependencies():
    """Check and install required dependencies"""
    required_packages = {
        'xlrd': 'xlrd>=2.0.1',
        'openpyxl': 'openpyxl>=3.0.0',
        'pandas': 'pandas>=1.0.0',
        'xlsxwriter': 'xlsxwriter>=1.0.0'
    }
    
    missing_packages = []
    for package, version in required_packages.items():
        if importlib.util.find_spec(package) is None:
            missing_packages.append(version)
    
    if missing_packages:
        print("Missing required packages. Installing...")
        try:
            import pip
            for package in missing_packages:
                pip.main(['install', package])
            print("Dependencies installed successfully!")
            importlib.reload(pd)
        except Exception as e:
            print(f"Error installing dependencies: {str(e)}")
            print("\nPlease install the following packages manually:")
            print("\npip install " + " ".join(missing_packages))
            sys.exit(1)

def read_excel_file(file_path):
    """
    Read Excel file with proper engine based on file extension
    """
    file_path_str = str(file_path)
    file_lower = file_path_str.lower()
    
    try:
        if file_lower.endswith('.xls'):
            return pd.read_excel(file_path_str, engine='xlrd', header=None)
        elif file_lower.endswith('.xlsx'):
            return pd.read_excel(file_path_str, engine='openpyxl', header=None)
        else:
            raise ValueError(f"Unsupported file format: {file_path_str}")
    except Exception as e:
        raise Exception(f"Error reading file {file_path_str}: {str(e)}")

def extract_data_from_excel_files(folder_path, output_format='xlsx'):
    """
    Extract data from Excel files in a folder and save to a new file.
    
    Args:
        folder_path (str): Path to the folder containing Excel files
        output_format (str): 'xlsx' or 'csv' for output format
        
    Returns:
        str: Path to the created output file
    """
    # Check dependencies first
    check_dependencies()
    
    results = []
    
    try:
        # Convert folder path to Path object for better handling
        folder = Path(folder_path)
        
        # Verify folder exists and is accessible
        if not folder.exists():
            raise Exception(f"Folder not found: {folder_path}")
        if not folder.is_dir():
            raise Exception(f"Path is not a directory: {folder_path}")
            
        # Get all Excel files in the folder and subfolders
        excel_files = []
        for ext in ['*.xls', '*.xlsx']:
            excel_files.extend(list(folder.rglob(ext)))
            
        if not excel_files:
            print("No Excel files found in the specified folder")
            return None
            
        print(f"Found {len(excel_files)} Excel files")
        
        for file_path in excel_files:
            print(f"Processing: {file_path.name}")
            try:
                # Read the Excel file
                df = read_excel_file(file_path)
                
                # Extract required cells (adjusting for 0-based indexing)
                company = df.iloc[14, 5] if df.shape[0] > 14 and df.shape[1] > 5 else None  # F15
                contact_name = df.iloc[16, 5] if df.shape[0] > 16 and df.shape[1] > 5 else None  # F17
                last_name = df.iloc[17, 5] if df.shape[0] > 17 and df.shape[1] > 5 else None  # F18
                email = df.iloc[19, 5] if df.shape[0] > 19 and df.shape[1] > 5 else None  # F20
                email2 = df.iloc[20, 5] if df.shape[0] > 20 and df.shape[1] > 5 else None  # F21
                
                results.append({
                    'File Name': file_path.name,
                    'Company': company,
                    'Contact Name': contact_name,
                    'Last Name': last_name,
                    'Email': email,
                    'Email2': email2
                })
                
            except Exception as e:
                print(f"Error processing {file_path.name}: {str(e)}")
                results.append({
                    'File Name': file_path.name,
                    'Company': f'Error: {str(e)}',
                    'Contact Name': 'Error',
                    'Last Name': 'Error',
                    'Email': 'Error',
                    'Email2': 'Error'
                })
    
    except Exception as e:
        print(f"Error accessing folder: {str(e)}")
        return None
    
    if not results:
        print("No data was extracted from any files")
        return None
        
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Generate output filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"extracted_data_{timestamp}"
    
    try:
        # Create output in current working directory
        if output_format.lower() == 'csv':
            output_path = f"{output_filename}.csv"
            results_df.to_csv(output_path, index=False)
        else:  # xlsx
            output_path = f"{output_filename}.xlsx"
            print(f"Creating Excel file: {output_path}")
            with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
                results_df.to_excel(writer, index=False, sheet_name='Extracted Data')
                
                # Get workbook and worksheet objects for formatting
                workbook = writer.book
                worksheet = writer.sheets['Extracted Data']
                
                # Add some formatting
                header_format = workbook.add_format({
                    'bold': True,
                    'bg_color': '#D3D3D3',
                    'border': 1
                })
                
                # Apply formatting to header row
                for col_num, value in enumerate(results_df.columns.values):
                    worksheet.write(0, col_num, value, header_format)
                    
                # Adjust column widths
                for idx, col in enumerate(results_df.columns):
                    max_length = max(
                        results_df[col].astype(str).apply(len).max(),
                        len(col)
                    ) + 2
                    worksheet.set_column(idx, idx, max_length)
        
        print(f"Data extracted successfully to: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"Error saving output file: {str(e)}")
        return None

# Usage example
if __name__ == "__main__":
    # First install required dependencies if needed
    check_dependencies()
    
    # Use raw string to handle Windows paths properly
    folder_path = os.path.expanduser(r"C:\Users\Laetitia\Downloads\JLquotations")
    
    print(f"Processing Excel files in: {folder_path}")
    
    # Create Excel file (default)
    print("\nCreating Excel output...")
    excel_output = extract_data_from_excel_files(folder_path)
    if excel_output:
        print(f"Excel file created: {excel_output}")

    # Optionally create CSV file
    print("\nCreating CSV output...")
    csv_output = extract_data_from_excel_files(folder_path, output_format='csv')
    if csv_output:
        print(f"CSV file created: {csv_output}")

Processing Excel files in: 'D:\\OneDrive - Kingsford (Thailand) Co., Ltd\\JLQuotations'

Creating Excel output...
Error accessing folder: Folder not found: 'D:\\OneDrive - Kingsford (Thailand) Co., Ltd\\JLQuotations'

Creating CSV output...
Error accessing folder: Folder not found: 'D:\\OneDrive - Kingsford (Thailand) Co., Ltd\\JLQuotations'
